In [32]:
import pandas as pd

mutations = pd.read_csv('merged_variants_anno.txt', sep="\t", header=0, index_col=False)
metasamples = pd.read_excel('../Metadata2.xlsx')

In [39]:
# Convert matrix to unique VAF value per cell (mean of VAFs)
import numpy as np

FILTER_DP = 25
FILTER_VAF = 5.0
FILTER_EFFECTS = ['chromosome', 
                  'duplication',
                  'inversion',
                  'exon_loss_variant',
                  'frameshift_variant',
                  'missense_variant',
                  'protein_protein_contact',
                  'structural_interaction_variant',
                  'splice_acceptor_variant',
                  'splice_donor_variant',
                  'stop_lost', 
                  'start_lost', 
                  'stop_gained']

def unify_vaf(str_vaf):
    if str_vaf in ['Na', 'NA']:
        return 0.0
    values = [float(c.split(':')[1].split(';')[2]) 
              if int(c.split(':')[1].split(';')[0]) >= FILTER_DP else 0.0 for c in str_vaf.split('|')]
    return np.round(np.mean(values), 2)

def unify_vaf_unfilter(str_vaf):
    if str_vaf in ['Na', 'NA']:
        return 0.0
    values = [float(c.split(':')[1].split(';')[2]) for c in str_vaf.split('|')]
    return np.round(np.mean(values), 2)

def get_maf(ref, alt):
    ref_len = len(ref)
    alt_len = len(alt)
    diff = ref_len - alt_len
    vtype = None
    if diff == 0:
        if alt_len == 1:
            vtype = 'SNP'
        elif alt_len == 2:
            vtype = 'DNP'
        elif alt_len == 3:
            vtype = 'TNP'
        else:
            vtype = 'ONP'
    elif diff > 0 and alt_len == 1:
        vtype = 'DEL'
    elif diff < 0 and ref_len == 1:
        vtype = 'INS'
    else:
        vtype = 'INDEL'
    return vtype


# Convert vcalls to a single VAF (no DP filter)
mutations_vaf = mutations.copy()
mutations_vaf.loc[:, mutations.columns[15:]] = mutations.loc[:, mutations.columns[15:]].applymap(unify_vaf_unfilter)
mutations_vaf.to_csv('merged_variants_anno_vaf.txt', sep="\t", index=False, header=True)

# Convert vcalls to a single VAF (with DP filter)
mutations_vaf = mutations.copy()
mutations_vaf.loc[:, mutations.columns[15:]] = mutations.loc[:, mutations.columns[15:]].applymap(unify_vaf)


# Create a MAF matrix (applying filters with GNOMAD filter)
to_keep = list()
maf_df = pd.DataFrame(columns=['Hugo_Symbol',
                               'Tumor_Sample_Barcode',
                               'Chromosome',
                               'Start_Position',
                               'Reference_Allele',
                               'Tumor_Seq_Allele2',
                               'Variant_Type'])
for index, row in mutations_vaf.iterrows():
    has_cosmic = row['COSMI70'] not in ['Na', '.']
    has_dbsnp = row['DBSNP'] not in ['Na', '.']
    has_gnomad = row['GNOMAD'] not in ['Na', '.']
    gnomad = float(row['GNOMAD']) if has_gnomad else 1.0
    if has_gnomad and gnomad >= 0.01:
        continue
    effect = row['EFFECT']
    vkey = row['VARIANT_KEY']
    gene = row['GENE']
    chrm, rest = vkey.split(':')
    start, rest2 = rest.split()
    ref, alt = rest2.split('>')
    start = int(start)
    if effect in FILTER_EFFECTS:
        to_keep.append(index)
        vtype = get_maf(ref, alt)
        if vtype == 'DEL':
            alt = '-' if len(alt) == 1 else alt[1:]
            ref = ref[1:]
            start += 1
        elif vtype == 'INS':
            ref = '-' if len(ref) == 1 else ref[1:]
            alt = alt[1:]
            start += 1
        for column, value in row[15:].items():
            if value >= FILTER_VAF:
                maf_df = maf_df.append({'Hugo_Symbol':gene,
                                        'Tumor_Sample_Barcode':column,
                                        'Chromosome':chrm,
                                        'Start_Position':start,
                                        'Reference_Allele':ref,
                                        'Tumor_Seq_Allele2':alt,
                                        'Variant_Type':vtype},
                                       ignore_index=True)
                
mutations_vaf = mutations_vaf.reindex(to_keep)
mutations_vaf.to_csv('merged_variants_anno_vaf_filtered.txt', sep="\t")

maf_df['Start_Position'] = pd.to_numeric(maf_df['Start_Position'])
maf_df.to_csv('merged_maf.txt', sep="\t")

In [26]:
import numpy as np
import pandas as pd

mutations = pd.read_csv('merged_variants_anno_vaf.txt', sep='\t', index_col=None, header=0)

genes = list()
with open('cancer_related_genes.txt', 'r') as f:
    for line in f.readlines():
        genes.append(line.strip())
        
mutations = mutations[mutations['GENE'].isin(genes)]

mutations.to_csv('merged_variants_anno_vaf_cancer_genes.txt', sep="\t", header=True)

True

In [40]:
import signatureanalyzer as sa

maf_df = pd.read_csv('merged_maf.txt', sep="\t", index_col=False, header=0)

_,spectra_snvs = sa.spectra.get_spectra_from_maf(maf_df, cosmic='cosmic3_exome', hgfile='hg38.2bit')
spectra_snvs.to_csv('merged_maf_spectra_snvs.txt', sep="\t", index=True, header=True)
_,spectra_indels = sa.spectra.get_spectra_from_maf(maf_df, cosmic='cosmic3_ID', hgfile='hg38.2bit')
spectra_indels.to_csv('merged_maf_spectra_indels.txt', sep="\t", index=True, header=True)

      * Mapping contexts: 932 / 29258

      * Mapping contexts: 1887 / 29258

      * Mapping contexts: 2926 / 29258

      * Mapping contexts: 4122 / 29258

      * Mapping contexts: 5393 / 29258

...

      * Mapping contexts: 27036 / 29258

      * Mapping contexts: 28263 / 29258

      * Mapping contexts: 29257 / 29258


      * Mapping contexts: 1547 / 2350

      * Mapping contexts: 2349 / 2350
